 ### Extract embeddings to be visualized

This code connects to the PostgreSQL database, reads the research_item table, and saves:

- in vectors.tsv — one embedding per line (float values separated by TAB, no header)
- in metadata.tsv — title and abstract for each embedding (with header)

In [ ]:
import psycopg2
import csv 
import os
import ast

conn = psycopg2.connect(
    dbname="scientilla",
    user="postgres",
    password="pwd",
    host="localhost",
    port=5444
)   
cur = conn.cursor()

out_dir = "projector"
os.makedirs(out_dir, exist_ok=True)

vectors_path = os.path.join(out_dir, "vectors.tsv")
metadata_path = os.path.join(out_dir, "metadata.tsv")

cur.execute("""
    SELECT
        data->>'title'    AS title,
        data->>'abstract' AS abstract,
        embedding::text   AS emb
    FROM research_item
    WHERE embedding IS NOT NULL
    ORDER BY id
    LIMIT 1000;
""")

rows = cur.fetchall()

with open(vectors_path, "w", encoding="utf-8") as vec_f, \
     open(metadata_path, "w", encoding="utf-8", newline="") as meta_f:

    meta_writer = csv.writer(meta_f, delimiter="\t")
    meta_writer.writerow(["title", "abstract"])

    for title, abstract, emb in rows:
        if emb is None:
            continue

        clean_title = (title or "").replace("\t", " ").replace("\n", " ")
        clean_abstract = (abstract or "").replace("\t", " ").replace("\n", " ")

        meta_writer.writerow([clean_title, clean_abstract])

        emb_list = ast.literal_eval(emb)

        vec_line = "\t".join(str(x) for x in emb_list)
        vec_f.write(vec_line + "\n")

cur.close()
conn.close()
